In [1]:
%matplotlib inline
from datetime import datetime, timedelta

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from IPython.display import display
import ipywidgets as widgets
import matplotlib.pyplot as plt
from metpy.units import units
from netCDF4 import num2date, Dataset
import numpy as np
from siphon.ncss import NCSS
import metpy.calc as mpcalc
import scipy.ndimage as ndimage

In [2]:
def getdata(y='',m=''):
    dt = datetime(y,m,1)
    
    # Grab Pressure Level Data
    hght_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                'NARR/pressure/hgt.{0:%Y%m}.nc'.format(dt))
    air_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                               'NARR/pressure/air.{0:%Y%m}.nc'.format(dt))
    shum_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                'NARR/pressure/shum.{0:%Y%m}.nc'.format(dt))
    uwnd_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                'NARR/pressure/uwnd.{0:%Y%m}.nc'.format(dt))
    vwnd_data = Dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/'
                                'NARR/pressure/vwnd.{0:%Y%m}.nc'.format(dt))
    
    return {'Geopotential_Height':hght_data.variables['hgt'],'Air_Temperature':air_data.variables['air'],
            'Specific_Humidity':shum_data.variables['shum'],'U_wind':uwnd_data.variables['uwnd'],
            'V_wind':vwnd_data.variables['vwnd'],'lat':hght_data.variables['lat'],'lon':hght_data.variables['lon'],
            'time':hght_data.variables['time'],'level':hght_data.variables['level']}

In [15]:
def plot(varname='Air_Temperature', level='850', day='19', hour='12'):
    data = x.widget.result
    year = x.widget.kwargs['y']
    month = x.widget.kwargs['m']

    # Pull out the lat and lon data
    lat = data['lat'][:]
    lon = data['lon'][:]

    plotcrs = ccrs.LambertConformal(central_latitude=45., central_longitude=-100.,
                                    standard_parallels=[30, 60])
    
    datacrs = ccrs.PlateCarree()

    tlatslons = plotcrs.transform_points(datacrs,lon,lat)
    tlon = tlatslons[:,:,0]
    tlat = tlatslons[:,:,1]
    
    dt = datetime(year,month,day,hour)
    vtimes = num2date(data['time'][:], units='hours since 1800-1-1 00:00:0.0')
    itime = np.where(vtimes==dt)[0][0]
    ilev = np.where(data['level'][:]==level)[0][0]

    hght = ndimage.gaussian_filter(data['Geopotential_Height'][itime,ilev,:,:].data, sigma=1.0)*units.meter
    uwnd = ndimage.gaussian_filter(data['U_wind'][itime,ilev,:,:], sigma=1.0)*units('m/s')
    vwnd = ndimage.gaussian_filter(data['V_wind'][itime,ilev,:,:], sigma=1.0)*units('m/s')
    
    if varname=='Wind_Speed':
        variable = mpcalc.get_wind_speed(uwnd, vwnd)
    elif varname=='Rel_Hum':
        shum = ndimage.gaussian_filter(data['Specific_Humidity'][itime,ilev,:,:], sigma=1.0)*units('dimensionless')
        tmpk = ndimage.gaussian_filter(data['Air_Temperature'][itime,ilev,:,:], sigma=1.0)*units('degK')
        e = mpcalc.vapor_pressure(level*units('hPa'), shum)
        es = mpcalc.saturation_vapor_pressure(tmpk)
        rel_hum = e/es*100
    elif (varname=='vort') | (varname=='absvort'):
        dx, dy = mpcalc.lat_lon_grid_deltas(lon,lat)
        if varname=='vort':
            variable = mpcalc.vorticity(uwnd,vwnd,dx,dy,dim_order='yx')*1e5
        elif varname=='absvort':
            variable =  mpcalc.absolute_vorticity(uwnd,vwnd,dx,dy, lat*units('degree'), dim_order='yx')*1e5
    else:
        variable = ndimage.gaussian_filter(data[varname][itime,ilev,:,:].data, sigma=1.0)
    
    fig = plt.figure(figsize=(16, 14))
    ax = fig.add_subplot(111, projection=plotcrs)
    ax.set_extent([230., 290., 20., 55.], ccrs.PlateCarree())

    # Add state boundaries to plot
    ax.add_feature(cfeature.STATES.with_scale('50m'))

    # Add country borders to plot
    ax.add_feature(cfeature.BORDERS.with_scale('50m'))

    if level>925:
        cint = range(0,5000,10)
    elif (level<=925) & (level>600):
        cint = range(0,5000,30)
    elif (level<=600) & (level>400):
        cint = range(0,10000,60)
    else:
        cint = range(0,20000,120)
        
    # Contour based on variable chosen
    cs = ax.contour(tlon, tlat, hght, cint,
                    linewidths=2, colors='black')
    plt.clabel(cs, inline=True, fmt='%d')
        
    if varname=='Air_Temperature':
        variable = (variable * units.degK).to('degC')
        cf = ax.contourf(tlon, tlat, variable, range(-50,51,2), cmap=plt.cm.coolwarm, extend='both')
        cs = ax.contour(tlon, tlat, variable, range(-50,51,2), colors='darkgrey', linestyles='dotted', extend='both')
        plt.clabel(cs, fmt='%d')
        plt.title(r''+str(level)+'-hPa Heights (m), Temp (C), and Wind Barbs (kts)',
          loc='left')
    elif varname=='Wind_Speed':
        if level<=500:
            fint = range(50,211,20)
        else:
            fint = range(10,100,20)
        cf = ax.contourf(tlon, tlat, variable.to('kts'), fint, cmap=plt.cm.BuPu, extend='both')
        plt.title(r''+str(level)+'-hPa Heights (m), Wind Speed (kts), and Wind Barbs (kts)',
          loc='left')
    elif varname=='Rel_Hum':
        cf = ax.contourf(tlon, tlat, rel_hum, range(50,100,5), cmap=plt.cm.Greens, extend='max')
        plt.title(r''+str(level)+'-hPa Heights (m), Relative Humidity (%), and Wind Barbs (kts)',
          loc='left')
    elif (varname=='vort'):
        fint = range(-15,16)
        fline = 'PuOr_r'
        cf = ax.contourf(tlon, tlat, variable, fint, cmap=fline, extend='both')
        plt.title(r''+str(level)+'-hPa Heights (m), Vorticity ($1e5$ $s^{-1}$), and Wind Barbs (kts)',
                      loc='left')
    elif varname=='absvort':
        clevavor500 = list(range(-8, 1, 1))+list(range(8, 49, 2))
        colors1 = plt.cm.YlOrRd(np.linspace(0, 1, 31))
        colors2 = plt.cm.BuPu(np.linspace(0.5, 0.75, 8))
        colorsavor500 = np.vstack((colors2, (1, 1, 1, 1), colors1))
        cf = ax.contourf(tlon, tlat, variable, clevavor500, colors=colorsavor500, extend='both')
        plt.title(r''+str(level)+'-hPa Heights (m), Abs. Vorticity ($1e5$ $s^{-1}$), and Wind Barbs (kts)',
                  loc='left')
    
    cb = fig.colorbar(cf, orientation='horizontal',ax=ax,pad=0,aspect=50,extendrect='True')

    if varname == 'Air_Temperature':
        cb.set_label(r'$^{o}C$', size='large')
        
    # Plot Wind Barbs
    # Transform Vectors and plot wind barbs.
    ax.barbs(lon, lat, uwnd.to('kts').m, vwnd.to('kts').m, length=6,
             regrid_shape=20, pivot='middle',transform=ccrs.PlateCarree())

    
    plt.title('VALID: {}'.format(vtimes[itime]), loc='right')
    
    plt.show()



In [4]:
year_widget = widgets.Dropdown(
    options=list(range(1979,2017)),
    description='Year', alignment='center')

month_widget = widgets.Dropdown(
    options=list(range(1,13)),
    description='Month', alignment='center')


var_widget = widgets.Dropdown(
    options={'Temperature': 'Air_Temperature',
             'Relative Humidity': 'Rel_Hum',
             'Wind Speed': 'Wind_Speed',
             'Vorticity': 'vort',
             'Absolute Vorticity': 'absvort'},
    description='Variable', alignment='center')
day_widget = widgets.Dropdown(description='Day', options=list(range(1,32)))
hour_widget = widgets.Dropdown(description='Hour', options=[0,3,6,9,12,15,18,21])
level_widget = widgets.Dropdown(description='Level', options=[1000,925,850,700,500,300,250,200,100])



In [5]:
x = widgets.interact_manual(getdata,y=year_widget,m=month_widget)

interactive(children=(Dropdown(description='Year', options=(1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 19…

In [16]:
p = widgets.interact_manual(plot, varname=var_widget, level=level_widget, day=day_widget, hour=hour_widget)

interactive(children=(Dropdown(description='Variable', options={'Temperature': 'Air_Temperature', 'Relative Hu…